In [6]:
import os
from dotenv import load_dotenv
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

True

In [7]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [10]:
# define Tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

# define Agent
weather_agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    description="一个通过使用工具为用户提供天气信息的智能体。",
    system_message="你是一个乐于助人的AI智能助手。擅长使用工具解决任务。任务完成后，回复<stop autogenworkshop>",
    tools=[get_weather],
)


In [12]:
# Define a termination condition. End the conversation if a specific text is mentioned.
termination = TextMentionTermination("<stop autogenworkshop>")

# Define Team with type set to RoundRobinGroupChat
agent_team = RoundRobinGroupChat(participants=[weather_agent], termination_condition=termination, max_turns=None)


In [13]:
 # 1、run() method to run the team
result = await agent_team.run(task="上海的天气如何?")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='上海的天气如何?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=80, completion_tokens=14), content=[FunctionCall(id='call_IkD6SJSeFxv0U0zGjw2dn42g', arguments='{"city":"上海"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in 上海 is 73 degrees and Sunny.', call_id='call_IkD6SJSeFxv0U0zGjw2dn42g')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='weather_agent', models_usage=None, content='The weather in 上海 is 73 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=116, completion_tokens=20), content='上海的天气是73华氏度，晴天。<stop autogenworkshop>', type='TextMessage')], stop_reason="Text '<stop autogenworkshop>' mentioned")


In [14]:
# 2、run_stream()metho to run team
async for message in agent_team.run_stream(task="上海的天气如何?"):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='上海的天气如何?' type='TextMessage'
source='weather_agent' models_usage=RequestUsage(prompt_tokens=152, completion_tokens=14) content=[FunctionCall(id='call_fVU8yTz0FFUr2n2pQ1cXbH2y', arguments='{"city":"上海"}', name='get_weather')] type='ToolCallRequestEvent'
source='weather_agent' models_usage=None content=[FunctionExecutionResult(content='The weather in 上海 is 73 degrees and Sunny.', call_id='call_fVU8yTz0FFUr2n2pQ1cXbH2y')] type='ToolCallExecutionEvent'
source='weather_agent' models_usage=None content='The weather in 上海 is 73 degrees and Sunny.' type='ToolCallSummaryMessage'
source='weather_agent' models_usage=RequestUsage(prompt_tokens=188, completion_tokens=20) content='上海的天气是73华氏度，晴天。<stop autogenworkshop>' type='TextMessage'
Stop Reason: Text '<stop autogenworkshop>' mentioned


In [15]:
# Method runs the team and uses the official Console tool to output in the proper format
# 3、run_stream()
stream = agent_team.run_stream(task="上海的天气如何?")
await Console(stream)

---------- user ----------
上海的天气如何?
---------- weather_agent ----------
[FunctionCall(id='call_92S53WcS9PogyHVq891FAxud', arguments='{"city":"上海"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in 上海 is 73 degrees and Sunny.', call_id='call_92S53WcS9PogyHVq891FAxud')]
---------- weather_agent ----------
The weather in 上海 is 73 degrees and Sunny.
---------- weather_agent ----------
上海的天气是73华氏度，晴天。<stop autogenworkshop>


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='上海的天气如何?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=224, completion_tokens=14), content=[FunctionCall(id='call_92S53WcS9PogyHVq891FAxud', arguments='{"city":"上海"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in 上海 is 73 degrees and Sunny.', call_id='call_92S53WcS9PogyHVq891FAxud')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='weather_agent', models_usage=None, content='The weather in 上海 is 73 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=260, completion_tokens=20), content='上海的天气是73华氏度，晴天。<stop autogenworkshop>', type='TextMessage')], stop_reason="Text '<stop autogenworkshop>' mentioned")